In [0]:
%sql
CREATE TABLE IF NOT EXISTS IDENTIFIER("tests." || :schema || ".transactions") (
    transaction_id      BIGINT NOT NULL PRIMARY KEY,
    transaction_date    DATE NOT NULL,
    transaction_time    TIMESTAMP NOT NULL,
    amount              DECIMAL(10,2),
    amount_with_tax     DECIMAL(10,2),
    description         STRING NOT NULL
)
CLUSTER BY (transaction_date)
TBLPROPERTIES (
    'delta.appendOnly' = true,
    'delta.enableChangeDataFeed' = true,
    'delta.enableRowTracking' = true
);

In [0]:
from pyspark.sql.functions import expr
schema = dbutils.widgets.get("schema")

# 1) Read from Bronze table (Delta) as a streaming source
#    This will pick up new rows as they arrive in the bronze table.
bronzeStreamDF = (
    spark
    .readStream
    .format("delta")
    .table(f"tests.bronze_source.transactions")
    .withColumn("amount_with_tax", expr("amount + (amount * tax)").cast("decimal(10,2)"))
    .drop("tax")
)

In [0]:
#dbutils.fs.rm(f"/Volumes/tests/{schema}/checkpoints", True)

In [0]:
# 2) Write the streaming data to the Silver table in append mode
silverStream = (
    bronzeStreamDF
    .writeStream
    .outputMode("append")
    .option("checkpointLocation", f"/Volumes/tests/{schema}/checkpoints")
    .trigger(availableNow=True)
    .toTable(f"tests.{schema}.transactions")
)